# SF Ballot Data

## Data Sources

### Department of Elections eData

https://www.sfelections.org/tools/election_data/dataset.php?ATAB=d2022-11-08

Generally, descriptive data on candidates, precincts, contests (all of labels)

### CVR (Cast Vote Record) Data

https://sfelections.sfgov.org/november-8-2022-election-results-detailed-reports

We have 22,193 json dumps which contain several ballots in each dump.  Important parts of each ballot:

* `PrecinctPortionId` -  Join to `PrecinctPortionManifest.json` to get precinct description.  Join decription with neighborhood `breakdown.xlxs` from other data source above for zip codes & neighborhoods.
    
* `BallotTypeId` - Join to `BallotTypeManifest.json` to get description.  (Not actually sure what this informs)

* List of `cards`, each of which contains a list of:
    * List of `contests`. Join to `ContestManifest.json` to interpret the issue voted on. Contents per contest:
        * `CandidateManifest.json` for candidates
        * `PartyManifest.json` for party
        * `OutstackConditionManifest.json` describes for write-ins / undervotes / overvotes / ambiguous / blank-ballot
        * List of candidates ranked, 1 being highest priority.  (Only 2 contests ask for >1 vote, "member BoE" & "member community college board"
    
## Framework

1) Pull in data, flesh out some descriptive language above, and have raw pandas dataframes below in a format more useful than JSON dumps
2) Create some flattened data sources
    1) Single ballots.  Column for every contest.  Annotate with precint id, tabulator id, ballot-type id.  Segment out votes with 'problems'
    2) ...
3) Join flattened data against all the manifests.  Join against neighborhoods
4) Analysis
    1) Check against existing records to validate load
    2) Contest D & E votes vs neighborhood analysis (There's turn-out percentages by neighborhood provided, but it's an excel sheet that looks meh to parse)
    

In [214]:
import pandas as pd
import numpy as np
import glob
import json
import tqdm.notebook as tqdm

In [7]:
# Web browser incantation to style so we can use the full screen space
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [63]:
# info on the ~21 ballot items 
measures_df = pd.read_csv('./measures.txt',sep='\t',encoding='utf-16')
measures_df.head(5)

,iMeasureID,sDesignation,szMeasureAbbr1,szMeasureAbbr2,iBallotPosition1,iBallotPosition2,dtDateReceived,szReceivedBy,szRemarks,szFilerTitle,...,sStyle,szLegalNoticePubDesc,szDeliveryMethodDesc,szGroupHdg,szBallotHeading,szSubHeading,sDistrictID,szDistrictName,szElectionDesc,dtElectionDate
0,100,A,Retiree Supplemental Cost Of Living Adjustment...,NaN,1,2,8/12/2022,NaN,NaN,NaN,...,NaN,NaN,NaN,MEASURES SUBMITTED TO THE VOTERS,CITY AND COUNTY PROPOSITIONS,NaN,*0,County Wide,November 8 2022 Consolidated General Election,11/8/2022
1,101,B,"Public Works Department And Commission, Sanita...",NaN,1,2,8/12/2022,NaN,NaN,NaN,...,NaN,NaN,NaN,MEASURES SUBMITTED TO THE VOTERS,CITY AND COUNTY PROPOSITIONS,NaN,*0,County Wide,November 8 2022 Consolidated General Election,11/8/2022
2,102,C,Homelessness Oversight Commission,NaN,1,2,8/12/2022,NaN,NaN,NaN,...,NaN,NaN,NaN,MEASURES SUBMITTED TO THE VOTERS,CITY AND COUNTY PROPOSITIONS,NaN,*0,County Wide,November 8 2022 Consolidated General Election,11/8/2022
3,103,D,Affordable Housing - Initiative Petition,NaN,1,2,8/12/2022,NaN,NaN,NaN,...,NaN,NaN,NaN,MEASURES SUBMITTED TO THE VOTERS,CITY AND COUNTY PROPOSITIONS,NaN,*0,County Wide,November 8 2022 Consolidated General Election,11/8/2022
4,104,E,Affordable Housing - Board Of Supervisors,NaN,1,2,8/12/2022,NaN,NaN,NaN,...,NaN,NaN,NaN,MEASURES SUBMITTED TO THE VOTERS,CITY AND COUNTY PROPOSITIONS,NaN,*0,County Wide,November 8 2022 Consolidated General Election,11/8/2022


In [64]:
# neighborhood and zip breakdowns
breakdown_df = pd.read_excel('./breakdown.xlsx')
breakdown_df.head(5)

,sZip,consolidation,szVotingPrecinctName,MailBallotPrecinct,iBalType,Assembly,BART,Congressional,Senatorial,Supervisorial,Neighborhood
0,94127.0,1101,Pct 1101,No,1,19,8,11,11,11,OCEANVIEW/MERCED/INGLESIDE
1,94132.0,1101,Pct 1101,No,1,19,8,11,11,11,OCEANVIEW/MERCED/INGLESIDE
2,94132.0,1102,Pct 1102,No,1,19,8,11,11,11,OCEANVIEW/MERCED/INGLESIDE
3,94132.0,1103,Pct 1103 MB,Yes,1,19,8,11,11,11,OCEANVIEW/MERCED/INGLESIDE
4,94132.0,1104,Pct 1104,No,2,19,8,15,11,11,OCEANVIEW/MERCED/INGLESIDE


## Start CRV Data Decoding

In [70]:
# All of the manifest files in the CRV data have the same JSON structure.
# Note that the BallotTypeContestManifest.json doesn't fit the schema, 
# looks like its a mapping of the contests on a given ballot type
def load_manifest(file_path):
    with open(file_path) as f:
        res=json.load(f)
    manifest_df = pd.DataFrame(res['List'])
    manifest_df.set_index('Id', inplace=True)
    return manifest_df

In [163]:
# What types of Contests are to be voted on given a ballot type?
ballot_type_contest_manifest_df = load_manifest('./CVR_Export/BallotTypeManifest.json')
ballot_type_contest_manifest_df.head(5)

,Description,ExternalId
Id,,
1,Ballot Type 23 - VBM,
2,Ballot Type 27 - VBM,
3,Ballot Type 26 - VBM,
4,Ballot Type 1 - VBM,
5,Ballot Type 19 - VBM,


In [78]:
candidate_manifest_df = load_manifest('./CVR_Export/CandidateManifest.json')
candidate_manifest_df.head(5)

,Description,ExternalId,ContestId,Type,Disabled
Id,,,,,
1,BRIAN DAHLE,,1,Regular,0
2,GAVIN NEWSOM,,1,Regular,0
3,ANGELA E. UNDERWOOD JACOBS,,2,Regular,0
4,ELENI KOUNALAKIS,,2,Regular,0
5,ROB BERNOSKY,,3,Regular,0


In [127]:
contest_manifest_df = load_manifest('./CVR_Export/ContestManifest.json')
contest_manifest_df

,Description,ExternalId,DistrictId,VoteFor,NumOfRanks,Disabled
Id,,,,,,
1,GOVERNOR,4000,101,1,0,0
2,LIEUTENANT GOVERNOR,4010,101,1,0,0
3,SECRETARY OF STATE,4020,101,1,0,0
4,CONTROLLER,4030,101,1,0,0
5,TREASURER,4040,101,1,0,0
...,...,...,...,...,...,...
57,Measure J,109,116,1,0,0
59,Measure L,111,116,1,0,0
60,Measure M,112,116,1,0,0


In [80]:
outstack_condition_manifest_df = load_manifest('./CVR_Export/OutstackConditionManifest.json')
outstack_condition_manifest_df.head(5)

,Description
Id,
0,Ambiguous
1,Writein
2,BlankBallot
5,Overvote
4,Undervote


In [71]:
cvr_precinct_portion_manifest_df = load_manifest('./CVR_Export/PrecinctPortionManifest.json')
cvr_precinct_portion_manifest_df.head(5)

,Description,ExternalId,PrecinctId
Id,,,
1,PCT 1101,1101-1,1
2,PCT 1102,1102-1,2
3,PCT 1103 MB,1103-1,3
4,PCT 1104,1104-2,4
5,PCT 1105,1105-2,5


In [72]:
# Tabulator ID to locations names
cvr_tabulator_df = load_manifest('./CVR_Export/TabulatorManifest.json')
cvr_tabulator_df.head(5)

,Description,VotingLocationNumber,VotingLocationName,ExternalId,Type,ThresholdMin,ThresholdMax,WriteThresholdMin,WriteThresholdMax
Id,,,,,,,,,
1,ICC01 Election Day,1,City Hall Vote Center,,ImagecastCentral,5,25,5,25
2,ICC01 Vote by Mail,1,City Hall Vote Center,,ImagecastCentral,5,25,5,25
3,ICC02 Election Day,1,City Hall Vote Center,,ImagecastCentral,5,25,5,25
4,ICC02 Vote by Mail,1,City Hall Vote Center,,ImagecastCentral,5,25,5,25
5,ICC03 Election Day,1,City Hall Vote Center,,ImagecastCentral,5,25,5,25


In [76]:
# What types of Contests are to be voted on given a ballot type?
with open('./CVR_Export/BallotTypeContestManifest.json') as f:
    res=json.load(f)
ballot_type_contest_manifest_df = pd.DataFrame(res['List'])
print(len(ballot_type_contest_manifest_df))
ballot_type_contest_manifest_df

3748


,BallotTypeId,ContestId
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
3743,80,68
3744,80,59
3745,80,60
3746,80,61


In [212]:
# These functions help unpack the per-ballot nested contest structure into something flat for our df
def marks_to_winner(marks):
    if not marks:
        return np.nan
    for mark in marks:
        if mark['Rank'] == 1:
            return mark['CandidateId']
#     raise RuntimeError(f"No 1st rank selected\n{marks}")
    return np.nan
    
def raw_ballot_to_flat(b):
    tabulator_id = b['TabulatorId']
    orig_ballot = b['Original']
    flat = {}
    
    # For each paper card scanned, mark the winner for the contests (each contest will be a column)
    for card in orig_ballot['Cards']:
        flat.update({f'contest_{contest["Id"]}': marks_to_winner(contest['Marks']) for contest in card['Contests']})
    
    flat.update({
        'precinct_portion_id': orig_ballot['PrecinctPortionId'],
        'ballot_type_id': orig_ballot['BallotTypeId'],
        'key_in_id': orig_ballot['Cards'][0]['KeyInId'],
        'tabulator_id': tabulator_id
    })
    return flat

In [181]:
with open('./CVR_Export/CvrExport_0.json') as f:
    res=json.load(f)
crv_export_df = pd.DataFrame(res['Sessions'])

In [211]:
crv_export_df.loc[0].Original['Cards']

[{'Id': 54860,
  'KeyInId': 54860,
  'PaperIndex': 0,
  'Contests': [{'Id': 1,
    'ManifestationId': 81779,
    'Undervotes': 0,
    'Overvotes': 0,
    'OutstackConditionIds': [],
    'Marks': [{'CandidateId': 2,
      'ManifestationId': 252694,
      'PartyId': 1,
      'Rank': 1,
      'MarkDensity': 0,
      'IsAmbiguous': False,
      'IsVote': True,
      'OutstackConditionIds': []}]},
   {'Id': 2,
    'ManifestationId': 81780,
    'Undervotes': 0,
    'Overvotes': 0,
    'OutstackConditionIds': [],
    'Marks': [{'CandidateId': 4,
      'ManifestationId': 252696,
      'PartyId': 1,
      'Rank': 1,
      'MarkDensity': 0,
      'IsAmbiguous': False,
      'IsVote': True,
      'OutstackConditionIds': []}]},
   {'Id': 3,
    'ManifestationId': 81781,
    'Undervotes': 0,
    'Overvotes': 0,
    'OutstackConditionIds': [],
    'Marks': [{'CandidateId': 6,
      'ManifestationId': 252698,
      'PartyId': 1,
      'Rank': 1,
      'MarkDensity': 0,
      'IsAmbiguous': False,
   

In [221]:
dfs = []
for file_path in tqdm.tqdm(glob.glob('./CVR_Export/CvrExport_*.json')):
    with open(file_path, 'r') as f:
        res = json.load(f)
    crv_export_df = pd.DataFrame(res['Sessions'])
    dfs.append(crv_export_df.apply(lambda x: raw_ballot_to_flat(x), axis=1, result_type='expand').set_index('key_in_id'))

raw_ballot_contest_results = pd.concat(dfs)

  0%|          | 0/22194 [00:00<?, ?it/s]

In [222]:
col_sort_key = lambda xs: [0 if not x.startswith('contest_') else int(x.split('_')[1]) for x in xs]

ballot_contest_results_annotated = raw_ballot_contest_results.copy()
ballot_contest_results_annotated['precint_desc'] = ballot_contest_results_annotated['precinct_portion_id'].apply(
    lambda x: cvr_precinct_portion_manifest_df.loc[x,'Description'])
ballot_contest_results_annotated['ballot_type_desc'] = ballot_contest_results_annotated['ballot_type_id'].apply(
    lambda x: ballot_type_contest_manifest_df.loc[x,'Description'])
ballot_contest_results_annotated['tabulator_desc'] = ballot_contest_results_annotated['tabulator_id'].apply(
    lambda x: cvr_tabulator_df.loc[x,'Description'])

# Sort columns and rename
ballot_contest_results_annotated.sort_index(axis=1, key=col_sort_key, inplace=True)
ballot_contest_results_annotated.columns = [
    contest_manifest_df.loc[int(x.split('_')[1]), 'Description'] if 
    x.startswith('contest_') else 
    x for x in ballot_contest_results_annotated.columns
]
ballot_contest_results_annotated

,tabulator_desc,precint_desc,ballot_type_desc,tabulator_id,ballot_type_id,precinct_portion_id,GOVERNOR,LIEUTENANT GOVERNOR,SECRETARY OF STATE,CONTROLLER,...,Measure E,Measure F,Measure G,Measure H,Measure I,Measure J,Measure L,Measure M,Measure N,Measure O
key_in_id,,,,,,,,,,,,,,,,,,,,,
60445.0,ICC04 Vote by Mail,PCT 7849,Ballot Type 16 - VBM,8.0,57.0,274.0,2.0,4.0,6.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62589.0,ICC04 Vote by Mail,PCT 9446,Ballot Type 24 - VBM,8.0,69.0,453.0,NaN,NaN,NaN,NaN,...,132.0,134.0,136.0,138.0,139.0,142.0,146.0,148.0,149.0,152.0
54326.0,ICC04 Vote by Mail,PCT 1132,Ballot Type 3 - VBM,8.0,27.0,29.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54325.0,ICC04 Vote by Mail,PCT 1132,Ballot Type 3 - VBM,8.0,27.0,29.0,2.0,4.0,6.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62229.0,ICC04 Vote by Mail,PCT 9426,Ballot Type 24 - VBM,8.0,69.0,435.0,NaN,NaN,NaN,NaN,...,132.0,133.0,135.0,137.0,139.0,142.0,145.0,148.0,150.0,152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58688.0,ICC12 Vote by Mail,PCT 7528,Ballot Type 10 - POLL,24.0,52.0,165.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58685.0,ICC12 Vote by Mail,PCT 7528,Ballot Type 10 - POLL,24.0,52.0,165.0,2.0,4.0,6.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58687.0,ICC12 Vote by Mail,PCT 7528,Ballot Type 10 - POLL,24.0,52.0,165.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
# Can compare governor results against summary here:
#    https://sfelections.sfgov.org/november-8-2022-election-results-detailed-reports
# Seems we're off by a bit
ballot_contest_results_annotated.groupby('GOVERNOR').count()

,tabulator_desc,precint_desc,ballot_type_desc,tabulator_id,ballot_type_id,precinct_portion_id,LIEUTENANT GOVERNOR,SECRETARY OF STATE,CONTROLLER,TREASURER,...,Measure E,Measure F,Measure G,Measure H,Measure I,Measure J,Measure L,Measure M,Measure N,Measure O
GOVERNOR,,,,,,,,,,,,,,,,,,,,,
1.0,44221,44221,44221,44221,44221,44221,42508,42495,42764,42607,...,454,456,469,444,464,457,468,471,447,462
2.0,257286,257286,257286,257286,257286,257286,247452,247882,245816,244960,...,3389,3468,3539,3347,3500,3403,3505,3517,3368,3423


## Why our data might look wrong (it's close though)

Comparing against reported data [here](https://www.sfelections.org/results/20221108/data/20221201/summary.pdf)

Looking at just the case for the Governor.  Seeing opposite signs and similar magnitude makes me think we just need to look at the amended ballot info (right now we're just pulling down the original.  The total diff has us over-reporting, possible that there's some metadata in the outstack conditions that might invalidate a vote (and we could pull that in).  


|              | **Newsome** | **Dahle**  | **Total**  |
|--------------|-------------|------------|------------|
| **Actual**   | 257,402     | 44,064     | 301,466    |
| **Our Data** | 257,286     | 44,221     | 301,507    |
| **Diff**     | -116        | 157        | 41         |

In [227]:
44221-44064

157